# 3. Data processing: goal_times

~~~
THIS NEEDS TO BE REFACTORED
~~~

**date**
: 2021-04-11

**desc**
: The goal times in the source are comma separated `str`.
An algorithm has been included before each part of the process.
We use list and string comprenhension to process the data.

**in**
: "data/out/epl_1819_game_id.csv"

**out**
: "data/out/goal_times.csv"

In [1]:
import pandas as pd
import numpy as np
from setup.data import Data
from setup.list_comprehension import ListComprehension

In [2]:
# read data in
df_in = Data.load_epl_1819_game_id()

In [3]:
# replace NaN values with "NaN"
df_in["home_team_goal_timings"].fillna(value="NaN", inplace=True)
df_in["away_team_goal_timings"].fillna(value="NaN", inplace=True)

In [4]:
df_in.head()

,total_goal_count,home_team_goal_timings,away_team_goal_timings,game_id
0,3,"3,83",90'2,0
1,3,11,"8,18",1
2,2,"24,90'1",NaN,2
3,2,NaN,"41,79",3
4,3,NaN,"34,45,80",4


In [5]:
# construct three lists for processing
game_id: list = df_in["game_id"].to_list()
home_timings: list = df_in["home_team_goal_timings"].to_list()
away_timings: list = df_in["away_team_goal_timings"].to_list()

## Step 1

Transfrom: `list(str) -> list(list(str))`

In [6]:
help(ListComprehension.transform_str_to_list_str)

Help on function transform_str_to_list_str in module setup.list_comprehension:

transform_str_to_list_str(a_list: list) -> list
    For each str in the list, if the string is not nan
    then separate the comma separated str to a list of strings.
    
    @param, a_list: list(str)
        a list of strings, where each string contains a comma
        separated string of goal times.
        Example: ["3,10,85", "5, 64", "23"]
    
    @returns, list(list(str))
        a list of lists of strings
        Example: [["3", "10", "85"], ["5", "64"], ["23"]]



In [7]:
# process home goals
home_goal_times: list = ListComprehension.transform_str_to_list_str(
    home_timings
)

In [8]:
# process away goals
away_goal_times: list = ListComprehension.transform_str_to_list_str(
    away_timings
)

## Step 2

We recast the `string` numerals to `int`.

Transform: `list(list(str)) -> list(list(int))`

In [9]:
help(ListComprehension.recast_numeral_to_int)

Help on function recast_numeral_to_int in module setup.list_comprehension:

recast_numeral_to_int(lists: list) -> list
    For each list in lists, and then for each numeral in the list,
    if the string is not NaN, then recast the numeral to an int.
    
    @param, list_strings, list(list(str))
        list of lists(str), where each element of the sub-list is
        a numeral representing the match time when a goal was
        scored.
        Example: [["3", "10", "85"], ["5", "64"], ["23"]]
    
    @returns, list(list(int))
        Example: [[3, 10, 85], [5, 64], [23]]



In [10]:
# process home
int_home_goal_times = ListComprehension.recast_numeral_to_int(
    home_goal_times
)

In [11]:
# process away
int_away_goal_times = ListComprehension.recast_numeral_to_int(
    away_goal_times
)

## Step 3

Each list in the lists Home and Away are linked, with one describing the home team and the other the away team.

Here we merge the two x lists of lists of ints into 1 x lists of lists of ints.

In [12]:
help(ListComprehension.merge_lists)

Help on function merge_lists in module setup.list_comprehension:

merge_lists(first_lists: list, second_lists: list) -> list
    Merges the elements of lists1 and lists2.
    
    @param, first_lists, list(list(int))
        Each element of list is a list of ints, where each elem is
        an int representing the match time when a goal was scored.
        Example: [[3, 10, 85], [5, 64]]
    
    @param, second_lists, list(list(int))
        Same description.
        Example: [[25], [79]]
    
    @returns, list(list(int))
        a combined list of lists of ints
        Example: [[3, 10, 85, 25], [5, 64, 79]]



In [13]:
merged_goal_times = ListComprehension.merge_lists(
    int_home_goal_times,
    int_away_goal_times
)

## Step 4

Sort each list of goal times into ascending order.

In [14]:
help(ListComprehension.sort_lists)

Help on function sort_lists in module setup.list_comprehension:

sort_lists(lists: list) -> list
    Orders each sub-list of a list of lists in ascending order.
    
    @param, lists, list(list(int))
        a list of lists of unsorted ints
        Example: [[85, 25], [5, 64, 35]]
    
    @returns, list(list(int))
        a list of lists of sorted ints
        Example: [[25, 85], [5, 35, 64]]



In [15]:
sorted_goal_times = ListComprehension.sort_lists(
    merged_goal_times
)

## Step 5

Created two partner lists.

One will be a `list(int)`, where each `int` is a goal time.
The second will be a goal's `game_id`.

They are partnered in the sense that elem `i` in both lists will be related.

In [16]:
help(ListComprehension.create_linked_lists)

Help on function create_linked_lists in module setup.list_comprehension:

create_linked_lists(lists: list) -> tuple
    Returns a tuple of linked lists. They are linked in the sense
    that elem `i` in both lists are related.
    
    @param, lists, list(list(int))
        a list of lists of sorted ints
        Example: [[25, 85], [5, 35, 64]]
    
    @returns, (list(int), list(int))
        tuple of lists of ints



In [17]:
game_ids, goal_times = ListComprehension.create_linked_lists(
    sorted_goal_times
)

## Step 6

We now creates a `DataFrame` of `games_ids, goal_times` and output to a csv file.

In [18]:
df_out = pd.DataFrame()

In [19]:
# add the game_ids and goal_times
df_out["game_ids"] = game_ids
df_out["goal_times"] = goal_times

In [20]:
# output file
df_out.to_csv(Data.PATH_OUT + "goal_times.csv", index=False)